In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [3]:
pair = "EUR_USD"
granularity = "M5"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

In [4]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [6]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2018-01-02 03:15:00+00:00,1.20154,1.20194,1.20154,1.20194,1.201224,1.201373
1,2018-01-02 03:20:00+00:00,1.20192,1.20203,1.20174,1.20198,1.201261,1.201392
2,2018-01-02 03:25:00+00:00,1.20196,1.20212,1.20196,1.20204,1.201306,1.201406
3,2018-01-02 03:30:00+00:00,1.20201,1.20202,1.20188,1.20192,1.201349,1.201419
4,2018-01-02 03:35:00+00:00,1.20190,1.20194,1.20174,1.20192,1.201403,1.201437


In [7]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [8]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [9]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
6,2018-01-02 03:45:00+00:00,1.20188,1.20188,1.20168,1.20170,1.201484,1.201471,0.000014,-0.000004,True
130,2018-01-02 14:05:00+00:00,1.20537,1.20576,1.20514,1.20560,1.206198,1.206226,-0.000028,0.000048,True
190,2018-01-02 19:05:00+00:00,1.20438,1.20480,1.20437,1.20470,1.204599,1.204589,0.000010,-0.000028,True
263,2018-01-03 01:15:00+00:00,1.20600,1.20602,1.20588,1.20588,1.205863,1.205865,-0.000002,0.000029,True
313,2018-01-03 05:25:00+00:00,1.20528,1.20550,1.20522,1.20550,1.205131,1.205072,0.000059,-0.000002,True


In [10]:
df_plot = df_ma.iloc[0:24].copy()

In [11]:
df_plot.shape

(24, 10)

In [12]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [15]:
df_all = pd.read_pickle("all_trades.pkl")
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,65992.000000,6.599200e+04,6.599200e+04,65992.000000,65992.000000,65992.000000,65992.000000,65992.000000,65992.000000
mean,32.162345,-3.359282e-04,4.074459e-04,-0.003182,-0.236192,-0.035059,11.773063,60.283307,49.809098
std,52.419874,1.671985e-02,1.673953e-02,1.000003,74.110465,74.110834,12.371754,62.578578,87.189995
min,0.607600,-3.192188e-01,-2.451250e-01,-1.000000,-1423.700000,-348.500000,4.000000,8.000000,1.000000
25%,0.915760,-1.993750e-04,-1.937500e-04,-1.000000,-22.300000,-28.200000,4.000000,16.000000,6.000000
50%,1.327360,-4.166667e-07,8.333333e-07,-1.000000,-0.400000,-10.400000,8.000000,32.000000,17.000000
75%,77.993000,1.910352e-04,1.968750e-04,1.000000,21.300000,7.600000,16.000000,96.000000,60.000000
max,165.324000,2.448125e-01,3.651250e-01,1.000000,1027.400000,1423.700000,64.000000,256.000000,1389.000000
